# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
#import geoviews.title_sources as gts

from holoviews import opts
from api_keys import geoapify_key

import geoviews.tile_sources as gts
import holoviews as hv
from holoviews import opts

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,afaahiti,-17.7500,-149.2833,27.93,82,45,6.45,PF,1709922697
1,1,murun-kuren,49.6342,100.1625,-13.58,87,66,0.86,MN,1709922698
2,2,west island,-12.1568,96.8225,27.99,89,100,4.63,CC,1709922698
3,3,mount gambier,-37.8333,140.7667,16.20,36,0,2.82,AU,1709922544
4,4,houghton,47.1219,-88.5690,4.82,68,0,0.89,US,1709922698
...,...,...,...,...,...,...,...,...,...,...
568,568,waddan,29.1614,16.1390,18.20,30,0,4.91,LY,1709922865
569,569,vorkuta,67.5000,64.0000,-28.05,99,59,2.71,RU,1709922865
570,570,campo gallo,-26.5833,-62.8500,40.61,26,40,6.23,AR,1709922866
571,571,midvagur,62.0511,-7.1939,5.46,75,75,8.23,FO,1709922866


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
# Display the map
# YOUR CODE HER
humidity_map = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size=city_data_df['Humidity']*0.1,
                                          hover_cols=['City'], color='Humidity', cmap='blues', alpha=0.6,
                                          tiles='OSM', frame_width=700, frame_height=500)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Drop any rows with null values
# YOUR CODE HERE
city_data_df=city_data_df.dropna()
# Display sample data
# YOUR CODE HERE
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,afaahiti,-17.7500,-149.2833,27.93,82,45,6.45,PF,1709922697
1,1,murun-kuren,49.6342,100.1625,-13.58,87,66,0.86,MN,1709922698
2,2,west island,-12.1568,96.8225,27.99,89,100,4.63,CC,1709922698
3,3,mount gambier,-37.8333,140.7667,16.20,36,0,2.82,AU,1709922544
4,4,houghton,47.1219,-88.5690,4.82,68,0,0.89,US,1709922698
...,...,...,...,...,...,...,...,...,...,...
568,568,waddan,29.1614,16.1390,18.20,30,0,4.91,LY,1709922865
569,569,vorkuta,67.5000,64.0000,-28.05,99,59,2.71,RU,1709922865
570,570,campo gallo,-26.5833,-62.8500,40.61,26,40,6.23,AR,1709922866
571,571,midvagur,62.0511,-7.1939,5.46,75,75,8.23,FO,1709922866


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,afaahiti,PF,-17.7500,-149.2833,82,
1,murun-kuren,MN,49.6342,100.1625,87,
2,west island,CC,-12.1568,96.8225,89,
3,mount gambier,AU,-37.8333,140.7667,36,
4,houghton,US,47.1219,-88.5690,68,
...,...,...,...,...,...,...
568,waddan,LY,29.1614,16.1390,30,
569,vorkuta,RU,67.5000,64.0000,99,
570,campo gallo,AR,-26.5833,-62.8500,26,
571,midvagur,FO,62.0511,-7.1939,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
import requests
import pandas as pd


# Base URL for the Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

# Print a message to indicate the start of the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Extract latitude and longitude for the current city
    lat, lng = row["Lat"], row["Lng"]
    
    # Set up the parameters for the Geoapify API request
 
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{lng},{lat},10000",
        "limit": 1,
        "bias": f"proximity:{lng},{lat}",
        "apiKey": geoapify_key
    }
    
    
    # Make the API request
    name_address = requests.get(base_url, params=params)
    # Convert the response to JSON format
    name_address = name_address.json()
    
    # Attempt to extract the first hotel's name from the response
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, log a message and continue
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    #Log the search results
    print(f"{hotel_df.loc[index,'City']}-nearest hotel:{hotel_df.loc[index,'Hotel Name']}")

# Display sample data
print(hotel_df.head())


Starting hotel search
afaahiti-nearest hotel:Omati Lodge
murun-kuren-nearest hotel:No hotel found
west island-nearest hotel:Cocos Village Bungalows
mount gambier-nearest hotel:The Old Mount Gambier Gaol
houghton-nearest hotel:Quality Inn & Suites
stephenville-nearest hotel:No hotel found
klyuchi-nearest hotel:No hotel found
port-aux-francais-nearest hotel:Keravel
atafu village-nearest hotel:No hotel found
dilling-nearest hotel:No hotel found
kirensk-nearest hotel:Ивушка
magadan-nearest hotel:Гостиница «ВМ-Центр»
puerto natales-nearest hotel:Dorotea Patagonia Hostel
touros-nearest hotel:Pousada Atlântico
akjoujt-nearest hotel:Hotel Fatima فندق فاطمة
st. john's-nearest hotel:Alt Hotel
st. paul-nearest hotel:Crowne Plaza
cambria-nearest hotel:Olallieberry Inn
waren-nearest hotel:Am Brauhaus
petropavlovsk-kamchatsky-nearest hotel:ООО Постоялый двор
ghanzi-nearest hotel:Ghanzi farmhouse
nar'yan-mar-nearest hotel:Гостиница "Агат"
nova sintra-nearest hotel:Residência Ka Dencho
adamstown-neare

zhosaly-nearest hotel:Gostinitsa Yeseta
saipan-nearest hotel:Chalan Kanoa Beach Hotel
essaouira-nearest hotel:Essaouira Wind Palace
fort lupton-nearest hotel:No hotel found
las tablas-nearest hotel:Hotel Don Jesus
san pedro de uraba-nearest hotel:No hotel found
kruisfontein-nearest hotel:No hotel found
nagqu-nearest hotel:Naqu Hotel
igrim-nearest hotel:Факел
avarua-nearest hotel:Paradise Inn
guanica-nearest hotel:Copamarina Beach Resort
kirakira-nearest hotel:No hotel found
nemuro-nearest hotel:お宿エクハシ
tidjikja-nearest hotel:No hotel found
georgetown-nearest hotel:Page 63 hostel
tura-nearest hotel:Hotel Polo Orchid
yerofey pavlovich-nearest hotel:No hotel found
illizi-nearest hotel:بوناقة
constantia-nearest hotel:Hotel Chérica
praia da vitoria-nearest hotel:Salles
bengkulu-nearest hotel:Dena Hotel
al jawf-nearest hotel:No hotel found
bruchmuhlbach-miesau-nearest hotel:Hotel Landgut Jungfleisch
grand-lahou-nearest hotel:Cap Lahou
lubang-nearest hotel:No hotel found
chonchi-nearest hotel:

santa rosalia-nearest hotel:Sol y Mar
nema-nearest hotel:No hotel found
isangel-nearest hotel:Tanna Lodge
newman-nearest hotel:No hotel found
maintirano-nearest hotel:Hotel Safari
north platte-nearest hotel:Howard Johnson Inn - North Platte
delcevo-nearest hotel:Хотел Македонија
ulaangom-nearest hotel:АНДУУД
calangute-nearest hotel:Hotel A Canoa
ascope-nearest hotel:Hostal Mario
bafoulabe-nearest hotel:No hotel found
tutoia-nearest hotel:Pousada San Vicente
waiblingen-nearest hotel:Adler
china-nearest hotel:No hotel found
grenaa-nearest hotel:Kystvejens Hotel og Konferencecenter
athens-nearest hotel:Eridanus
bijar-nearest hotel:خانه معلم
vodnyy-nearest hotel:No hotel found
labrador city-nearest hotel:Carol Inn
tupaciguara-nearest hotel:Macauba
uray-nearest hotel:Турсунт
lompoc-nearest hotel:Red Roof Inn Lompoc
margate-nearest hotel:The Reading Rooms
hami-nearest hotel:Хами
lorengau-nearest hotel:Seeadler Bay Hotel
torghay-nearest hotel:No hotel found
jiwani-nearest hotel:No hotel found

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display
#Create the points plot
map_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, size=hotel_df['Humidity']*0.1,
                                  hover_cols=['Hotel Name', 'Country'],  # Add hotel name and country to hover
                                  color='Humidity', cmap='blues', alpha=0.5)

# Configure the map plot
# YOUR CODE HERE
final_plot = map_plot * gts.CartoLight()
final_plot.opts(
    gv.opts.Points(width=700, height=500, tools=['hover'], size=8, color='Humidity', cmap='blues', alpha=0.5),
    gv.opts.WMTS(width=700, height=500))  # Options for the tile source

# YOUR CODE HERE
final_plot

:Overlay
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,Hotel Name,Country)
   .WMTS.I   :WMTS   [Longitude,Latitude]